# 1. Imports and setup

In [12]:
#Import libraries

import matplotlib.pyplot as plt
import rasterio
from shapely.geometry import box
from rasterio import plot
from rasterio import features
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.transform import Affine
from PIL import Image, ImageDraw
import numpy as np
import os
import torch
import torch.nn as nn
import json
import xarray as xr
import rioxarray as rio
import cv2
import geopandas as gpd

torch.random.manual_seed(0)

In [2]:
!nvidia-smi

Thu Oct  9 09:21:25 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   50C    P8              36W / 120W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 2. Convertion from files to desired input files

## 2.1 Json to binary mask - first version to get preprocessed images

In [3]:
inputDir = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\MasksToPNGMasks"
mainOutput = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\MasksOutput"

In [7]:
def json_to_mask(json_path, output_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_width = data.get('imageWidth')
    image_height = data.get('imageHeight')
    if image_width is None or image_height is None:
        raise ValueError("Las dimensiones de la imagen no están especificadas en el archivo JSON.")

    mask = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask)

    for shape in data['shapes']:
        if (shape['label'] == "cropArea") or (shape['label'] == "CropArea") or (shape['label'] == "CropLand") or (shape['label'] == "cropLand"):    
            points = shape['points']
            points_int = [(int(x), int(y)) for x, y in points]
            draw.polygon(points_int, outline=255, fill=255)

    # Obtener el nombre base del archivo JSON sin extensión
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    # Crear el nombre del archivo de salida con el sufijo "_m.png"
    output_filename = f"{base_name}_m.png"
    # Unir la ruta del directorio de salida con el nombre del archivo
    output_path = os.path.join(output_dir, output_filename)

    # Guardar la máscara como imagen PNG
    mask.save(output_path)

In [10]:
# Loop through all .tif files in input_dir
for file in os.listdir(inputDir):
    jsonPath = os.path.join(inputDir, file)
    # Call the custom function to generate tiles and PNGs
    json_to_mask(jsonPath, mainOutput)

## 2.2 Process files from AI4boundaries Dataset

From nc in date 6 (early Northern Hemisphere autumn), to single image

In [8]:
inputDS = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestToProcess"
OutputDS = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestProcessed"

In [34]:
inputCat = "C:\\Users\\Leonardo\\Desktop\\CATALONIA\\IMGS\\NCS"
OutputCat = "C:\\Users\\Leonardo\\\Desktop\\CATALONIA\\PROC"

In [ ]:
#Se usan las imagenes de AT, NL y SI de AI4boundaries

def procesar_nc_a_tif(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for archivo in os.listdir(input_dir):
        if archivo.endswith(".nc"):
            ruta_nc = os.path.join(input_dir, archivo)
            nombre_base = os.path.splitext(archivo)[0]
            ruta_tif = os.path.join(output_dir, f"{nombre_base}.tif")

            ds = xr.open_dataset(ruta_nc, decode_coords="all")

            # Seleccionar la ultima fecha
            ds_fecha0 = ds.isel(time=5)

            # Establecer las dimensiones espaciales
            ds_fecha0 = ds_fecha0.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

            # Verificar si el CRS está presente
            if ds_fecha0 is None:
                continue

            if ds_fecha0.rio is None:
                continue

            try:
                crs = ds_fecha0.rio.crs
                # Si el CRS es None o string vacío, omitir archivo
                if crs is None or str(crs) == "" or str(crs).lower() == "none":
                    #print(f"Archivo {archivo} sin CRS válido. Se omite.")
                    continue
            except Exception as e:
                print(f"Error leyendo CRS en {archivo}: {e}. Se omite.")
                continue
            
            for var in ds_fecha0.data_vars:
                if 'grid_mapping' in ds_fecha0[var].attrs:
                    del ds_fecha0[var].attrs['grid_mapping']

            # Seleccionar las bandas deseadas
            bandas = ['B4', 'B3', 'B2', 'B8', 'NDVI']  # Rojo, Verde, Azul, NIR, NDVI
            bandas_disponibles = [banda for banda in bandas if banda in ds_fecha0.variables]

            if not bandas_disponibles:
                print(f"No se encontraron las bandas especificadas en {archivo}.")
                continue

            # Crear una lista de DataArrays para las bandas seleccionadas
            dataarrays = []
            for banda in bandas_disponibles:
                da = ds_fecha0[banda]
                dataarrays.append(da)

            # Combinar las bandas en un solo DataArray multibanda
            da_combinado = xr.concat(dataarrays, dim="band")

            # Guardar como GeoTIFF
            da_combinado.rio.to_raster(ruta_tif)


In [ ]:
procesar_nc_a_tif(inputDS, OutputDS)

In [ ]:
#Se usan las imagenes de Cataluña (ES) - Simular epoca seca en Andes

os.makedirs(OutputCat, exist_ok=True)

for archivo in os.listdir(inputCat):
    if not archivo.endswith(".nc"):
        continue

    ruta_nc = os.path.join(inputCat, archivo)
    nombre_base = os.path.splitext(archivo)[0]
    ruta_tif = os.path.join(OutputCat, f"{nombre_base}.tif")

    ds = xr.open_dataset(ruta_nc, decode_coords="all")

    if "time" not in ds.dims or len(ds["time"]) <= 5:
        print(f"{archivo}: no tiene suficientes fechas")
        continue

    ds_fecha0 = ds.isel(time=5)

    # Detectar nombres de coordenadas
    x_name = "x" if "x" in ds_fecha0.dims else "lon"
    y_name = "y" if "y" in ds_fecha0.dims else "lat"
    ds_fecha0 = ds_fecha0.rio.set_spatial_dims(x_dim=x_name, y_dim=y_name, inplace=False)

    if not ds_fecha0.rio.crs:
        ds_fecha0 = ds_fecha0.rio.write_crs("EPSG:3035", inplace=True)

    # Eliminar grid_mapping
    for var in ds_fecha0.data_vars:
        ds_fecha0[var].attrs.pop("grid_mapping", None)

    bandas = ['B4','B3','B2','B8','NDVI']
    bandas_disponibles = [b for b in bandas if b in ds_fecha0.data_vars]

    if not bandas_disponibles:
        print(f"{archivo}: sin bandas válidas")
        continue

    da_combinado = xr.concat([ds_fecha0[b] for b in bandas_disponibles], dim="band")
    da_combinado.rio.to_raster(ruta_tif)

De imagen en tamaño 256 por 256, a 512 por 512

In [5]:
# 256 a 512
from skimage.transform import resize
from skimage.util import img_as_uint

def escalar_tiff_uint16(carpeta_entrada, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_archivo in os.listdir(carpeta_entrada):
        if nombre_archivo.lower().endswith((".tif", ".tiff")):
            ruta_entrada = os.path.join(carpeta_entrada, nombre_archivo)
            nombre_base = os.path.splitext(nombre_archivo)[0]
            ruta_salida = os.path.join(carpeta_salida, f"{nombre_base}_512.tif")

            with rasterio.open(ruta_entrada) as src:
                # Leer todas las bandas
                bandas = src.read()
                num_bandas, altura, anchura = bandas.shape
                
                # Redimensionar cada banda
                bandas_redimensionadas = np.empty((num_bandas, 512, 512), dtype=np.uint16)
                for i in range(num_bandas):
                    banda = bandas[i]
                    banda = banda.astype(np.float32)
                    banda = np.nan_to_num(banda, nan=0.0, posinf=0.0, neginf=0.0)

                    # Reemplazar valores < 0 por 0
                    banda[banda < 0] = 0

                    # Normalizar la banda a rango [0, 1] para resize
                    banda_normalizada = banda / 65535.0
                    banda_redimensionada = resize(
                        banda_normalizada,
                        (512, 512),
                        mode='reflect',
                        anti_aliasing=True
                    )
                    # Convertir de nuevo a uint16
                    bandas_redimensionadas[i] = img_as_uint(banda_redimensionada)

                # Crear perfil para el nuevo archivo
                perfil = src.profile
                perfil.update({
                    'driver': 'GTiff',
                    'count': num_bandas,
                    'dtype': 'uint16',
                    'width': 512,
                    'height': 512,
                    'nodata': 0,
                    'transform': rasterio.transform.from_origin(
                        src.transform.c,  # x origin
                        src.transform.f,  # y origin
                        src.transform.a * (anchura / 512),  # pixel width
                        src.transform.e * (altura / 512)   # pixel height
                    )
                })

                # Guardar la imagen redimensionada
                with rasterio.open(ruta_salida, 'w', **perfil) as dst:
                    dst.write(bandas_redimensionadas)


In [ ]:
carpeta_entrada = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestProcessed"
carpeta_salida = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\Test256To512"
escalar_tiff_uint16(carpeta_entrada, carpeta_salida)

Máscara binaria para archivos tiff en las máscaras (tambien, otro reescalado previo, imputando valores bajos y NaNs)

In [5]:
def convertir_tiff_a_mascara_binaria(carpeta_entrada, carpeta_salida, umbral=127):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre in os.listdir(carpeta_entrada):
        if nombre.lower().endswith((".tif", ".tiff")):
            ruta_tif = os.path.join(carpeta_entrada, nombre)
            nombre_salida = os.path.splitext(nombre)[0] + "_m.png"
            ruta_png = os.path.join(carpeta_salida, nombre_salida)

            with rasterio.open(ruta_tif) as src:
                banda = src.read(1)  # Solo primera banda

                # Convertir en máscara binaria: valores > umbral a blanco (255), el resto negro (0)
                mascara = np.where(banda > umbral, 255, 0).astype(np.uint8)

                # Guardar como PNG
                Image.fromarray(mascara).save(ruta_png)


In [7]:
carpeta_entrada = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TrainMasksToProcess"
carpeta_salida1 = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TrainMasks256To512"
carpeta_salida2 = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TrainMasksProcessed"

In [6]:
def escalar_tiff_uint16_2(carpeta_entrada, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_archivo in os.listdir(carpeta_entrada):
        if nombre_archivo.lower().endswith((".tif", ".tiff")):
            ruta_entrada = os.path.join(carpeta_entrada, nombre_archivo)
            nombre_base = os.path.splitext(nombre_archivo)[0]
            ruta_salida = os.path.join(carpeta_salida, f"{nombre_base}_512.tif")

            with rasterio.open(ruta_entrada) as src:
                bandas = src.read()
                num_bandas, altura, anchura = bandas.shape
                
                bandas_redimensionadas = np.empty((num_bandas, 512, 512), dtype=np.uint16)

                for i in range(num_bandas):
                    banda = bandas[i].astype(np.float32)

                    # Imputar 0 en NaN o valores vacíos
                    banda = np.nan_to_num(banda, nan=0.0)

                    # Recorte entre percentiles 2 y 98
                    p2, p98 = np.percentile(banda, (2, 98))
                    banda = np.clip(banda, p2, p98)

                    # Normalizar entre 0 y 1
                    rango = p98 - p2
                    if rango == 0:
                        banda_norm = np.zeros_like(banda)
                    else:
                        banda_norm = (banda - p2) / rango

                    # Redimensionar
                    banda_redim = resize(
                        banda_norm,
                        (512, 512),
                        mode='reflect',
                        anti_aliasing=True
                    )

                    # Convertir a uint16
                    banda_uint16 = np.clip(banda_redim * 65535, 0, 65535).astype(np.uint16)
                    bandas_redimensionadas[i] = banda_uint16

                perfil = src.profile
                perfil.update({
                    'driver': 'GTiff',
                    'count': num_bandas,
                    'dtype': 'uint16',
                    'width': 512,
                    'height': 512,
                    'nodata': 0,
                    'transform': rasterio.transform.from_origin(
                        src.transform.c,
                        src.transform.f,
                        src.transform.a * (anchura / 512),
                        src.transform.e * (altura / 512)
                    )
                })

                with rasterio.open(ruta_salida, 'w', **perfil) as dst:
                    dst.write(bandas_redimensionadas)

In [8]:
escalar_tiff_uint16_2(carpeta_entrada, carpeta_salida1)

In [9]:
convertir_tiff_a_mascara_binaria(carpeta_salida1, carpeta_salida2, umbral=0)

In [ ]:
#Delete images with "no masks" (aka images where white pixels are less than 10% of total)
# This should be outputed to a new directory, taking as input the directory with the binary masks:
def delete_no_masks(maskInput, maskOutput, umbral=0.1):
    os.makedirs(maskOutput, exist_ok=True)

    for name in os.listdir(maskInput):
        if name.lower().endswith((".png", ".jpg", ".jpeg")):
            path_mask = os.path.join(maskInput, name)
            mask = Image.open(path_mask).convert("L")
            mask_np = np.array(mask)

            # Calculate the percentage of white pixels
            total_pixels = mask_np.size
            white_pixels = np.sum(mask_np > 0)
            white_percentage = white_pixels / total_pixels

            if white_percentage >= umbral:
                # Save the mask to the output directory
                mask.save(os.path.join(maskOutput, name))


In [10]:
masksWB = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TrainMasksProcessed"
masksNV = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TrainMasksProcessedNonEmpty"

In [6]:
delete_no_masks(masksWB, masksNV, umbral=0.1)

Here we delete the ndvi band from the input tiles

In [37]:
trainToSelect = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\train"
trainSelected = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\trainSelected"
trainTif = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\trainTif"
train256To512 = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\train256To512"
train4bands = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\train4bands"
trainfoo = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\trainfoo"

In [21]:
#select images with masks, comparing names in trainToSelect dir (actual files) and the mask directory:
def select_images_with_masks(train_dir, mask_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    # Define a loop in the mask directory, select the name of the png file, then search in train dir and copy to trainSelected dir:
    # we shall remove the _512_m.png suffix and then copy the rest of the name with the .nc extension:
    for mask_file in os.listdir(mask_dir):
        if mask_file.endswith("_512_m.png"):
            # Remove the suffix to get the base name
            base_name = mask_file[:-10]  # Remove "_512_m.png"
            train_file = base_name + ".nc"  # Assuming the train files are .nc

            # Check if the corresponding train file exists
            if os.path.exists(os.path.join(train_dir, train_file)):
                # Copy the train file to the output directory
                src_path = os.path.join(train_dir, train_file)
                dst_path = os.path.join(output_dir, train_file)
                os.system(f'copy "{src_path}" "{dst_path}"')

In [22]:
select_images_with_masks(trainToSelect, masksNV, trainSelected)

In [38]:
procesar_nc_a_tif(trainSelected, trainfoo)

Error leyendo CRS en SE_20866_S2_10m_256.nc: Invalid projection: : (Internal Proj Error: proj_create: unrecognized format / unknown name). Se omite.
Error leyendo CRS en SE_22819_S2_10m_256.nc: Invalid projection: : (Internal Proj Error: proj_create: unrecognized format / unknown name). Se omite.


In [34]:
# Upsize 256 to 512 images
escalar_tiff_uint16(trainTif, train256To512)

In [6]:
def preserve_first_four_bands_batch(input_dir, output_dir):
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    # Iterate through all .tif files in the input directory
    for filename in os.listdir(input_dir):
        if filename.endswith(".tif"):
            input_path = os.path.join(input_dir, filename)
            output_path = os.path.join(output_dir, f"{os.path.splitext(filename)[0]}_f.tif")

            with rasterio.open(input_path) as src:
                if src.count < 4:
                    continue

                # Update metadata
                meta = src.meta.copy()
                meta.update(count=4)

                with rasterio.open(output_path, 'w', **meta) as dst:
                    for i in range(1, 5):
                        dst.write(src.read(i), i)

In [ ]:
input5bands = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundariesDS\\Input5bandsTrain"
input4bands = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundariesDS\\Input4bandsTrain"

In [35]:
#preserve_first_four_bands_batch(input5bands, input4bands)
preserve_first_four_bands_batch(train256To512, train4bands)

## 2.3 Mask generator usando Mapa de Superficie Agrícola

In [4]:
shapefile_path = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\SuperficieAgricola\\SuperficieAgricolaNacional.shp"
tiff_folder = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\SelectedSplits"
output_folder = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\TempMasks"

In [5]:
os.makedirs(output_folder, exist_ok=True)

In [8]:
gdf = gpd.read_file(shapefile_path)

In [31]:
black_th = 0.04
cloud_th = 0.25

In [ ]:
# Exclude clouds and padding

for AreaDir in os.listdir(tiff_folder):
    area_path = os.path.join(tiff_folder, AreaDir)
    if not os.path.isdir(area_path):
        continue

    # Create subfolder for masks
    area_output = os.path.join(output_folder, AreaDir)
    os.makedirs(area_output, exist_ok=True)

    for SubDir in os.listdir(area_path):
        if SubDir != "finalTiles":
            continue

        tile_dir = os.path.join(area_path, SubDir)
        for tif_name in os.listdir(tile_dir):
            if not tif_name.endswith(".tif"):
                continue

            tif_path = os.path.join(tile_dir, tif_name)
            with rasterio.open(tif_path) as src:
                img = src.read()  # shape: [bands, H, W]
                transform = src.transform
                out_shape = (src.height, src.width)
                bbox = src.bounds

                # --- CRS match ---
                if gdf.crs != src.crs:
                    gdf = gdf.to_crs(src.crs)

                # --- Crop shapefile ---
                raster_box = box(*bbox)
                gdf_clip = gdf[gdf.intersects(raster_box)].copy()

                # --- Rasterize agricultural mask ---
                if gdf_clip.empty:
                    agri_mask = np.zeros(out_shape, dtype=np.uint8)
                else:
                    shapes = ((geom, 1) for geom in gdf_clip.geometry)
                    agri_mask = features.rasterize(
                        shapes=shapes,
                        out_shape=out_shape,
                        transform=transform,
                        fill=0,
                        dtype=np.uint8
                    )

                # --- Read bands ---
                R, G, B, NIR = img[:4].astype(float)

                # Normalize if needed
                if R.max() > 1:
                    R, G, B, NIR = [x / 10000 for x in (R, G, B, NIR)]

                # --- Cloud detection ---
                cloud_mask = np.logical_and(B > cloud_th, NIR > cloud_th)

                # --- Padding detection (black borders) ---
                padding_mask = np.logical_and.reduce((R < black_th, G <  black_th, B <  black_th, NIR <  black_th))

                # --- Combine masks ---
                exclude_mask = np.logical_or(cloud_mask, padding_mask)
                final_mask = agri_mask * (~exclude_mask)

                # --- Save PNG ---
                mask_img = Image.fromarray(final_mask.astype(np.uint8) * 255)
                out_path = os.path.join(area_output, tif_name.replace("_p.tif", "_m.png"))
                mask_img.save(out_path)